<a href="https://colab.research.google.com/github/matiaspoullain/Herramientas-cuantitativas-Contaminacion-aire-y-vehiculos/blob/colabs/Notebooks/%20Notebook%20de%20Preprocesamiento_Vehiculos_CABA_y_Movilidad_Google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Descarga y prepocesamiento de los datos de circulación vehicular por peajes
Esta notebok contiene el código utilizado en la descarga y preprocesamiento de los datos relacionados con los vehículos que se utilizarán en las próximas notebook. En esta se descarga y preprocesa los datos de conteos vehiculares en los peajes de CABA, los datos de movilidad de población de CABA medidos por Google también los datos de venta de combustibles en CABA, todos públicos y de acceso libre. Sus fuentes respectivas son:
*    https://data.buenosaires.gob.ar/dataset/flujo-vehicular-por-unidades-peaje-ausa
*   https://www.google.com/covid19/mobility/
*   https://datos.gob.ar/dataset/energia-precios-volumenes-eess---resolucion-110404


##1. Datos de circulación vehicular por peajes AUSA, CABA

Cargo librería que se utilizará en esta notebook

In [ ]:
import pandas as pd
from google.colab import files

Descargo los datos

In [ ]:
!wget 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ausa/flujo-vehicular-por-unidades-peaje-ausa/flujo-vehicular-2020.csv'
!wget 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ausa/flujo-vehicular-por-unidades-de-peaje-ausa/flujo-vehicular-2019.csv'

--2022-02-01 18:28:28--  https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ausa/flujo-vehicular-por-unidades-peaje-ausa/flujo-vehicular-2020.csv
Resolving cdn.buenosaires.gob.ar (cdn.buenosaires.gob.ar)... 200.16.89.97
Connecting to cdn.buenosaires.gob.ar (cdn.buenosaires.gob.ar)|200.16.89.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274090244 (261M) [text/csv]
Saving to: ‘flujo-vehicular-2020.csv’

flujo-vehicular-202  94%[=================>  ] 246.00M   423KB/s    in 10m 1s  

2022-02-01 18:38:32 (419 KB/s) - Connection closed at byte 257944863. Retrying.

--2022-02-01 18:38:33--  (try: 2)  https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ausa/flujo-vehicular-por-unidades-peaje-ausa/flujo-vehicular-2020.csv
Connecting to cdn.buenosaires.gob.ar (cdn.buenosaires.gob.ar)|200.16.89.97|:443... connected.
HTTP request sent, awaiting response... 206 Partial Content
Length: 274090244 (261M), 16145381 (15M) remaining [text/csv]
Saving to: ‘flujo-vehicu

Leo los datos de 2019 y 2020. Luego realizo las transformaciones y agregaciones necesarias para obtener una tabla con las siguientes dos columnas: 


1.   _fecha_hora_: Fecha y hora del registro del pasaje de vehículos
2.   _cantidad_pasos_: Cantidad de vehículos que atravesaron un peaje en CABA durante la _fecha_hora_ explicitada



In [ ]:
datos = pd.DataFrame()

for i in ['2019', '2020']:
  datos_it = pd.read_csv('flujo-vehicular-' + i + '.csv')
  datos_it['fecha'] = datos_it['fecha'].astype(str).str[0:10]
  datos_it['fecha_hora'] = datos_it['fecha'] + ' ' + datos_it['hora_inicio'].astype(int).astype(str).str.zfill(2) + ':00:00'
  datos = datos.append(datos_it[['fecha_hora', 'cantidad_pasos']].groupby(["fecha_hora"], as_index = False).sum().round(), ignore_index = True)

datos['fecha_hora'] = pd.to_datetime(datos['fecha_hora'], format = '%Y-%m-%d %H:%M:%S')
datos

,fecha_hora,cantidad_pasos
0,2019-01-01 00:00:00,2275.0
1,2019-01-01 01:00:00,9389.0
2,2019-01-01 02:00:00,12809.0
3,2019-01-01 03:00:00,8796.0
4,2019-01-01 04:00:00,4863.0
...,...,...
17539,2020-12-31 19:00:00,14101.0
17540,2020-12-31 20:00:00,15301.0
17541,2020-12-31 21:00:00,10420.0
17542,2020-12-31 22:00:00,2821.0


Guardo la tabla resultante en formato CSV y la descargo

In [ ]:
datos.to_csv('flujo-vehicular-2019-2020.csv', index=False)
files.download('flujo-vehicular-2019-2020.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##2. Datos de movilidad de Google en CABA

Descargo el .ZIP conteniendo los datos de movilidad de todo el mundo y lo extraigo

In [ ]:
!wget 'https://www.gstatic.com/covid19/mobility/Region_Mobility_Report_CSVs.zip'
!mkdir '/movilidad_mundo'
!unzip 'Region_Mobility_Report_CSVs.zip' -d 'movilidad_mundo'

--2022-03-08 19:33:24--  https://www.gstatic.com/covid19/mobility/Region_Mobility_Report_CSVs.zip
Resolving www.gstatic.com (www.gstatic.com)... 173.194.213.94, 2607:f8b0:400c:c01::5e
Connecting to www.gstatic.com (www.gstatic.com)|173.194.213.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76161677 (73M) [application/zip]
Saving to: ‘Region_Mobility_Report_CSVs.zip’

Region_Mobility_Rep 100%[===================>]  72.63M  6.27MB/s    in 12s     

2022-03-08 19:33:36 (6.07 MB/s) - ‘Region_Mobility_Report_CSVs.zip’ saved [76161677/76161677]

Archive:  Region_Mobility_Report_CSVs.zip
  inflating: movilidad_mundo/2020_AE_Region_Mobility_Report.csv  
  inflating: movilidad_mundo/2020_AF_Region_Mobility_Report.csv  
  inflating: movilidad_mundo/2020_AG_Region_Mobility_Report.csv  
  inflating: movilidad_mundo/2020_AO_Region_Mobility_Report.csv  
  inflating: movilidad_mundo/2020_AR_Region_Mobility_Report.csv  
  inflating: movilidad_mundo/2020_AT_Region_Mobility

Me quedo solo con los datos de CABA

In [ ]:
movilidad = pd.read_csv("movilidad_mundo/2020_AR_Region_Mobility_Report.csv")
movilidad = movilidad[(movilidad['sub_region_1'] == 'Buenos Aires') & (pd.isna(movilidad['sub_region_2']))]
movilidad.reset_index(drop=True, inplace=True)
movilidad.drop(['country_region_code', 'country_region', 'sub_region_1', 'sub_region_2', 'metro_area', 'iso_3166_2_code', 'census_fips_code', 'place_id'], axis = 1, inplace = True)
movilidad

,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,2020-02-15,6.0,1.0,6.0,9.0,-2.0,0.0
1,2020-02-16,2.0,-2.0,-15.0,3.0,-9.0,2.0
2,2020-02-17,-9.0,-14.0,-27.0,2.0,4.0,2.0
3,2020-02-18,9.0,9.0,23.0,11.0,7.0,-1.0
4,2020-02-19,8.0,5.0,29.0,10.0,6.0,-2.0
...,...,...,...,...,...,...,...
316,2020-12-27,-47.0,-15.0,-21.0,-45.0,-5.0,9.0
317,2020-12-28,-38.0,2.0,-28.0,-47.0,-37.0,13.0
318,2020-12-29,-36.0,9.0,-13.0,-44.0,-37.0,12.0
319,2020-12-30,-35.0,27.0,-22.0,-45.0,-35.0,10.0


Guardo y descargo el CSV

In [ ]:
movilidad.to_csv('movilidad-buenos-aires.csv', index=False)
files.download('movilidad-buenos-aires.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Combustibles vendidos en CABA

In [ ]:
!wget http://datos.minem.gob.ar/dataset/708f9ab4-829b-4f02-b507-f303c5bc4800/resource/15067e62-47ad-4a26-a2c8-daeecb7ac043/download/precios-eess-2020.csv
!wget http://datos.minem.gob.ar/dataset/708f9ab4-829b-4f02-b507-f303c5bc4800/resource/51bda70a-199b-4499-84ff-76ef63c01040/download/precios-eess-2019.csv

--2022-03-08 19:33:45--  http://datos.minem.gob.ar/dataset/708f9ab4-829b-4f02-b507-f303c5bc4800/resource/15067e62-47ad-4a26-a2c8-daeecb7ac043/download/precios-eess-2020.csv
Resolving datos.minem.gob.ar (datos.minem.gob.ar)... 186.33.254.210
Connecting to datos.minem.gob.ar (datos.minem.gob.ar)|186.33.254.210|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46717761 (45M) [text/csv]
Saving to: ‘precios-eess-2020.csv’

precios-eess-2020.c 100%[===================>]  44.55M  14.2MB/s    in 3.4s    

2022-03-08 19:33:50 (13.0 MB/s) - ‘precios-eess-2020.csv’ saved [46717761/46717761]

--2022-03-08 19:33:50--  http://datos.minem.gob.ar/dataset/708f9ab4-829b-4f02-b507-f303c5bc4800/resource/51bda70a-199b-4499-84ff-76ef63c01040/download/precios-eess-2019.csv
Resolving datos.minem.gob.ar (datos.minem.gob.ar)... 186.33.254.210
Connecting to datos.minem.gob.ar (datos.minem.gob.ar)|186.33.254.210|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 469379

In [ ]:
combustibles = pd.DataFrame()

for i in ['2019', '2020']:
  df_it = pd.read_csv('precios-eess-' + i + '.csv')
  combustibles = combustibles.append(df_it, ignore_index = True)


combustibles = combustibles[combustibles['provincia'] == 'CAPITAL FEDERAL']
combustibles

,anio,mes,operador,nro_inscripcion,bandera,fecha_de_baja,cuit,tipo_negocio,direccion,localidad,provincia,producto,canal_de_comercializacion,precio_sin_impuestos,precio_con_impuestos,volumen,precio_surtidor,no_novimientos,exentos
451,2019,1,AGROGANADERA MODELO SOCIEDAD ANONIMA,7557,BLANCA,NaN,30-70829348-9,Distribuidor,LIMA 575,CAPITAL FEDERAL,CAPITAL FEDERAL,N/D,N/D,0.0000,0.0000,0.000,0.00,SI,NaN
591,2019,1,ALBERDI 2299 SRL,6270,SHELL C.A.P.S.A.,NaN,33-70934480-9,Estación de servicio,AV. JUAN B. ALBERDI 2299,CAPITAL FEDERAL,CAPITAL FEDERAL,Gas Oil Grado 2,Al público,22.9900,36.5800,12.009,36.58,NO,f
592,2019,1,ALBERDI 2299 SRL,6270,SHELL C.A.P.S.A.,NaN,33-70934480-9,Estación de servicio,AV. JUAN B. ALBERDI 2299,CAPITAL FEDERAL,CAPITAL FEDERAL,Nafta (premium) de más de 95 Ron,Al público,29.7820,43.9900,92.046,43.99,NO,f
593,2019,1,ALBERDI 2299 SRL,6270,SHELL C.A.P.S.A.,NaN,33-70934480-9,Estación de servicio,AV. JUAN B. ALBERDI 2299,CAPITAL FEDERAL,CAPITAL FEDERAL,Gas Oil Grado 3,Al público,27.8630,41.9800,32.107,41.98,NO,f
594,2019,1,ALBERDI 2299 SRL,6270,SHELL C.A.P.S.A.,NaN,33-70934480-9,Estación de servicio,AV. JUAN B. ALBERDI 2299,CAPITAL FEDERAL,CAPITAL FEDERAL,Nafta (súper) entre 92 y 95 Ron,Al público,22.8720,37.8710,126.467,37.76,NO,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494976,2020,12,YAPAY NEASA S.R.L.,7820,SHELL C.A.P.S.A.,NaN,30-69524367-3,Estación de servicio,AV. CORRIENTES N° 3101,CAPITAL FEDERAL,CAPITAL FEDERAL,Nafta (premium) de más de 95 Ron,Al público,50.8218,77.2968,120.290,79.45,NO,t
494977,2020,12,YAPAY NEASA S.R.L.,7820,SHELL C.A.P.S.A.,NaN,30-69524367-3,Estación de servicio,AV. CORRIENTES N° 3101,CAPITAL FEDERAL,CAPITAL FEDERAL,Gas Oil Grado 3,Al público,50.5391,72.7607,37.770,75.37,NO,t
494978,2020,12,YAPAY NEASA S.R.L.,2678,SHELL C.A.P.S.A.,NaN,30-69524367-3,Estación de servicio,AV. BALBIN 3560,CAPITAL FEDERAL,CAPITAL FEDERAL,Gas Oil Grado 2,Al público,42.9589,63.6016,10.450,65.31,NO,t
494979,2020,12,YAPAY NEASA S.R.L.,2678,SHELL C.A.P.S.A.,NaN,30-69524367-3,Estación de servicio,AV. BALBIN 3560,CAPITAL FEDERAL,CAPITAL FEDERAL,Gas Oil Grado 3,Al público,50.8994,73.2041,52.820,75.19,NO,t


Agrupo por mes y por tipo de combustible sumando volúmenes de combustible vendidos:

In [ ]:
combustibles = combustibles[['anio', 'mes', 'producto', 'volumen' ]].groupby(['anio', 'mes', 'producto'], as_index = False).sum()
combustibles = combustibles[combustibles['producto'].str.contains("Nafta") | combustibles['producto'].str.contains("GNC")]
combustibles

,anio,mes,producto,volumen
0,2019,1,GNC,1.373439e+07
6,2019,1,Nafta (premium) de más de 95 Ron,1.965211e+04
7,2019,1,Nafta (súper) entre 92 y 95 Ron,4.272068e+04
8,2019,2,GNC,1.304179e+07
14,2019,2,Nafta (premium) de más de 95 Ron,1.946279e+04
...,...,...,...,...
167,2020,11,Nafta (premium) de más de 95 Ron,1.852527e+04
168,2020,11,Nafta (súper) entre 92 y 95 Ron,3.464911e+04
169,2020,12,GNC,1.340952e+07
174,2020,12,Nafta (premium) de más de 95 Ron,2.281999e+04


Guardar y descargar el CSV de combustibles:

In [ ]:
combustibles.to_csv('combustibles.csv', index=False)
files.download('combustibles.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>